![](images/obspy_logo_full_524x179px.png)

<div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">Breve guida a ObsPy: gestione dei dati della forma d'onda</div>

Questa guida trae spunto dal Tutorial di:

* Lion Krischer ([@krischer](https://github.com/krischer))
* Tobias Megies ([@megies](https://github.com/megies))

Seismo-Live: http://seismo-live.org

---

In [ ]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

<img src="images/Stream_Trace.svg" width=90%>

* la lettura dei dati delle forme d'onda restituisce un oggetto **`Stream`**.

In [ ]:
from obspy import read
st = read("./data/waveform_PFO.mseed", format="mseed")
print(st)

- Il carattere `*` di UNIX può essere utilizzato per leggere più file contemporaneamente
- Rilevamento automatico del formato dei file, non è necessario preoccuparsi dei formati di file

   - formati attualmente supportati: **MiniSEED, SAC, SEG-Y, SEG-2, GSE1 / 2, Seisan, SH, DataMark, CSS, wav, y, Q, ASCII, Guralp Compressed Format, Kinemetrics EVT, K-NET e KiK, PDAS, Reftek 130, WIN (continua a crescere ...)**
   - sono inclusi nuovi formati ogni volta che viene fornita una routine di lettura di base (o, per esempio, documentazione sufficiente sulla compressione dei dati, ecc.)
   - È possibile aggiungere formati di file personalizzati specifici dell'utente (tramite plug-in) per essere rilevati automaticamente in un'installazione ObsPy locale

In [ ]:
from obspy import read
st = read("./data/waveform_*")
print(st)

- per i file MiniSEED, è supportata solo la lettura di brevi parti di file senza che i file vengano letti e tenuti interamente in memoria (consente di risparmiare tempo e memoria quando si lavora su grandi raccolte di file di grandi dimensioni)

In [ ]:
from obspy import UTCDateTime

t = UTCDateTime("2011-03-11T05:46:23.015400Z")
st = read("./data/waveform_*", starttime=t + 10 * 60, endtime=t + 12 * 60)
print(st)

#### L'oggetto Stream

 - Un oggetto Stream è una collezione di oggetti Trace

In [ ]:
from obspy import read
st = read("./data/waveform_PFO.mseed")
print(type(st))

In [ ]:
print(st)

In [ ]:
print(st.traces)

In [ ]:
print(st[0])

- Più tracce possono essere assemblate usando l'operatore **`+`** (o usando i metodi `.append()` e `.extend()`)

In [ ]:
st1 = read("./data/waveform_PFO.mseed")
st2 = read("./data/waveform_PFO_synthetics.mseed")

st = st1 + st2
print(st)

st3 = read("./data/waveform_BFO_BHE.sac")

st += st3
print(st)

 - loop  su tracce comodo (e ben leggibile)

In [ ]:
for tr in st:
    print(tr.id)

 - Lo stream è utile per applicare la stessa elaborazione a un grande maggiore di forme d'onda diverse o per raggruppare tracce per l'elaborazione (ad esempio tre componenti di una stazione in uno stream)

#### L'oggetto Trace (traccia)

- un oggetto Trace è un blocco singolo contiguo di dati di forme d'onda (cioè serie temporali a intervalli regolari, senza spazi vuoti)
- un oggetto Trace contiene una quantità limitata di metadati in un oggetto simile al dizionario (come **`Trace.stats`**) che descrive completamente le serie temporali specificando..
   * posizione / strumento di registrazione (rete, stazione, posizione e codice canale)
   * ora di inizio
   * frequenza di campionamento

In [ ]:
st = read("./data/waveform_PFO.mseed")
tr = st[0]  # get the first Trace in the Stream
print(tr)

In [ ]:
print(tr.stats)

- Per le applicazioni personalizzate a volte è necessario manipolare direttamente i metadati di una Traccia.
- I metadati della Traccia rimarranno **coerenti**, poiché tutti i valori sono derivati dall'orario di inizio, i dati e la frequenza di campionamento e sono **aggiornati automaticamente**

In [ ]:
print(tr.stats.delta, "|", tr.stats.endtime)

In [ ]:
tr.stats.sampling_rate = 5.0
print(tr.stats.delta, "|", tr.stats.endtime)

In [ ]:
print(tr.stats.npts)

In [ ]:
tr.data = tr.data[:100]
print(tr.stats.npts, "|", tr.stats.endtime)

- Alcuni metodi rendono semplici le manipolazioni di base

In [ ]:
tr = read("./data/waveform_PFO.mseed")[0]
tr.plot()

In [ ]:
print(tr)
tr.resample(sampling_rate=100.0)
print(tr)

In [ ]:
print(tr)
tr.trim(tr.stats.starttime + 12 * 60, tr.stats.starttime + 14 * 60)
print(tr)
tr.plot()

In [ ]:
tr.detrend("linear")
tr.taper(max_percentage=0.05, type='cosine')
tr.filter("lowpass", freq=0.1)
tr.plot()

In [ ]:
# try tr.<Tab> for other methods defined for Trace
tr.detrend?

- Dati grezzi sono disponibili come [**`numpy.ndarray`**](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html) (come **`Trace.data`**)

In [ ]:
print(tr.data[:20])

- I dati possono essere modificati direttamente per esempio...

...facendo operazioni aritmetiche (veloce, gestito in C da NumPy)

In [ ]:
print(tr.data ** 2 + 0.5)

...usando [**`numpy.ndarray`** metodo predefinito](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html) (sempre gestito in C da NumPy)

In [ ]:
print(tr.data.max())
print(tr.data.mean())
print(tr.data.ptp())
# try tr.data.<Tab> per una lista di metodi numpy definiti su ndarray

...usando le funzioni **`numpy`** (sempre gestito in C da NumPy)

In [ ]:
import numpy as np
print(np.abs(tr.data))
# puoi provare np.<Tab> ma qui c'è molto
# try np.a<Tab>

...inserendo puntatori a routine esistenti C/Fortran dall'interno di Python!

Questo viene fatto internamente in diversi posti, ad es. per correlazioni incrociate, beamforming o in librerie di tipi di file di terze parti come ad es. libmseed.

- Gli oggetti traccia possono anche essere generati manualmente con i dati in un [**`numpy.ndarray`**] (http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html) (ad es. quando è necessario analizzare le forme d'onda da file ascii non standard)

In [ ]:
from obspy import Trace

x = np.random.randint(-100, 100, 500)
tr = Trace(data=x)
tr.stats.station = "XYZ"
tr.stats.starttime = UTCDateTime()

tr.plot()

- Ogetti Stream possono essere assemblati da Tracce generate manualmente.

In [ ]:
from obspy import Stream

tr2 = Trace(data=np.random.randint(-300, 100, 1000))
tr2.stats.starttime = UTCDateTime()
tr2.stats.sampling_rate = 10.0
st = Stream([tr, tr2])

st.plot()

#### Metodi pre-definiti in  **`Stream`** / **`Trace`**

- La maggior parte dei metodi che funziona su un oggetto Trace funziona anche per un oggetto Stream. Sono semplicemente eseguiti su ciascuna traccia. [Vedere la ducumentazione ObsPy per una panoramica dei metodi disponibili](http://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html) (oppure prova **`st.<Tab>`**).
 - **`st.filter()`** - Filtra tutte le tracce allegate.
 - **`st.trim()`** - taglia tutte le tracce.
 - **`st.resample()`** / **`st.decimate()`** - Cambia il passo di campionamento.
 - **`st.trigger()`** - Avvia un algoritmo di trigger.
 - **`st.plot()`** / **`st.spectrogram()`** - Visualizza i dati.
 - **`st.attach_response()`**/**`st.remove_response()`**, **`st.simulate()`** - Correzione per lo strumento
 - **`st.merge()`**, **`st.normalize()`**, **`st.detrend()`**, **`st.taper()`**, ...
- Un oggetto **`Stream`** può essere anche esportato in molti formati, quindi ObsPy può essere per converire tra diversi formati di file.

In [ ]:
st = read("./data/waveform_*.sac")
st.write("output_file.mseed", format="MSEED")
!ls -l output_file*

<img src="images/Stream_Trace.svg" width=90%>

#### Esercizi con le Trace
 - Crea un **`numpy.ndarray`** con tutti zero (ad esempio usa **`numpy.zeros()`**) e metti un impulso ideale da qualche parte in esso
  - Inizializza un oggetto **`Trace`** con il tuo array di dati
  - Inserisci alcune informazioni sulla stazione (ad es. Rete, stazione, ..)
  - Stampa il riepilogo della traccia e fai un plot la traccia
  - Modifica la frequenza di campionamento a 20 Hz
  - Modifica l'ora di inizio della traccia all'ora di inizio di questa sessione
  - Stampa il riepilogo traccia e fai di nuovo un plot la traccia

In [ ]:
x = np.zeros(300)
x[100] = 1.0
tr = Trace(data=x)
tr.stats.station = "ABC"
tr.plot()
tr.stats.sampling_rate = 20.0
tr.stats.starttime = UTCDateTime(2018, 4, 13, 15, 0, 0)
print(tr)
tr.plot()

- Usa **`tr.filter(...)`** e applica un filtro passa-basso con una frequanza d'angolo di 1 Hertz.
- Visualizza il grafico di anteprima, ci sono alcuni secondi di zeri che possiamo tagliare.

In [ ]:
tr.filter("lowpass", freq=1)
tr.plot()

- Usa **`tr.trim(...)`** per rimuovere alcuni degli zero all'inizio ed alla fine
- visualizza di nuovo il grafico

In [ ]:
tr.trim(tr.stats.starttime + 3, tr.stats.endtime - 5)
tr.plot()

- Scala le ampiezze della traccia di un fattore 500
- Aggiungi rumore standard gaussiano alla traccia (usa [**`np.random.randn()`**](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randn.html))
- Visualizza di nuovo la traccia

In [ ]:
tr.data = tr.data * 500
tr.data = tr.data + np.random.randn(len(tr))
tr.plot()

#### Esercizi con gli Stream

- Leggi tutti i dati di terremoto di Tohoku in un oggetto Stream ("./data/waveform\_\*")
- Stampa il riepilogo del flusso

In [ ]:
st = read("./data/waveform_*")
print(st)

- Usa **`st.select()`** per tenere solo le tracce della stazione BFO nello stream. 
- Visualizza le tracce

In [ ]:
st = st.select(station="BFO")
st.plot()

- trim the data to a 10 minute time window around the first arrival (just roughly looking at the preview plot)
- display the preview plot and spectrograms for the stream (with logarithmic frequency scale, use `wlen=50` for the spectrogram plot)

- Taglia i dati in una finestra temporale di 10 minuti intorno al primo arrivo (guardando solo approssimativamente la figura precedente)
- visualizza le tracce e gli spettrogrammi per lo stream (con scala di frequenza logaritmica, usa `wlen=50` per il grafico dello spettrogramma)

In [ ]:
t1 = UTCDateTime(2011, 3, 11, 5, 55)
st.trim(t1, t1 + 10 * 60)
st.plot()
st.spectrogram(log=True, wlen=50);

- rimuovi l'andamento lineare dai dati, applica un tapering e un filtro passa-basso a 0,1 Hertz
- mostra di nuovo il grafico

In [ ]:
st.detrend("linear")
st.filter("lowpass", freq=0.1)
st.plot()